In [ ]:
from collections import Counter

def js_to_python(s):
    d = {
        "true": "True",
        "false": "False",
        "null": "None",
        "[]": "None",
        "][": "],["
    }
    for k, v in d.items():
        s = s.replace(k, v)
    return s

def read_line(s):
    try:
        return eval(js_to_python(s))
    except Exception as e:
        print(e)
        a, b = eval(js_to_python(s))
        return a

def listdecorator(f):
    return lambda x: list(f(x))

@listdecorator
def get_trial_data(exp_dicts):
    return filter(is_trial, exp_dicts)
        
def is_trial(d):
    st = 'stimulus_type'
    return (d[st] == 'test') if (st in d.keys()) else False

g0_instruction = '\n<b>Geben Sie in jeder Runde Ihr Bestes, um zu gewinnen und beenden Sie die Studie bitte erst, wenn Sie alle Runden beendet haben.</b>\n'
g1_instruction = '\n<b>Geben Sie in jeder Runde Ihr Bestes, um zu gewinnen, indem Sie versuchen\n    möglichst zufällig zu spielen, da dies die beste Strategie ist, um eine\n    künstliche Intelligenz zu besiegen.</b>\n'

def get_group(vp):
    return int(raw_data[vp][2]['stimulus'] == g1_instruction)

def nonempty_vp_idx(Ns, thr=0):
    return map(lambda x: x[0], (filter(lambda x: x[1] > thr, enumerate(Ns))))

def get_responses(vp_data):
    return list(map(lambda d: (d['response'], d['bot_response']), filter(lambda d: 'bot_response' in d.keys(), vp_data)))

def result(t):
    return 0 if t[0] == t[1] else -1 if t[0]+t[1] in 'RPSR' else 1

def count(vp):
    return Counter(map(result, get_responses(raw_data[vp]))) 

def compute_winrate(vp):
    r = count(vp)
    return r[1] / (r[-1] + r[1])

In [ ]:
from bogatron import draw_pdf_contours, Dirichlet, xy2bc, trimesh
# draw_pdf_contours(Dirichlet([200, 100, 100]))

In [ ]:
xy2bc((0,0)).sum()

In [ ]:
fn = '../data/experiment/jatos_results_20230119081857.txt'
fn = '../data/experiment/jatos_results_20230119120735.txt'
fn = '../data/experiment/jatos_results_20230120090949.txt'
fn = '../data/experiment/jatos_results_20230120094311.txt'
fn = '../data/experiment/jatos_results_20230125083535.txt'
fn = '../data/experiment/jatos_results_20230126102948.txt'
with open(fn) as fh:
    raw_data = [read_line(s) for s in fh]
    #raw_data = fh.readlines()
len(raw_data)

In [ ]:
vps = list(filter(lambda vp: len(raw_data[vp]) > 600, range(len(raw_data))))
len(vps)
#group = [int(raw_data[vp][2]['stimulus'] == g1_instruction) for vp in vps]
group = {vp: get_group(vp) for vp in vps}
sum(group.values()) / len(group)
# group

In [ ]:
strats = ['nash_equilibrium_strategy', 'winstay', 'rotate_strategy', 'learn_preference', 'dont_always_copy_opponent_move']
strats = ['super_male_strategy', 'nash_equilibrium_strategy', 'winstay', 'rotate_strategy', 'learn_preference', 'dont_always_copy_opponent_move']
for vp in vps:
    strategy = strats[list(map(lambda d: d['strategy'], filter(lambda d: 'strategy' in d.keys(), raw_data[vp])))[0]]
    print(f'VP {vp}: {compute_winrate(vp)}, Group {get_group(vp)}')

In [ ]:
[count(vp) for vp in vps if get_group(vp) == 0]

In [ ]:
[count(vp) for vp in vps if get_group(vp) == 1]

In [ ]:
for vp in vps:
    wr = compute_winrate(vp)
    
sum([compute_winrate(vp) for vp in vps]) / len(vps)

In [ ]:
sum([compute_winrate(vp) for vp in vps if group[vp]]) / sum(group.values())

In [ ]:
sum([compute_winrate(vp) for vp in vps if not group[vp]]) / (len(group) - sum(group.values()))

In [ ]:
Counter(map(result, get_responses(raw_data[vp])))[-1]

In [ ]:
import scipy.stats
# draw_pdf_contours(Dirichlet((1, 10, 1)))
# posterior = scipy.stats.dirichlet((58, 77, 65))
# draw_pdf_contours(posterior)

In [ ]:
import matplotlib.pyplot as plt
from collections import namedtuple

In [ ]:
Counts = namedtuple('Counts', ['win', 'lose', 'draw'])

In [ ]:
c = Counts(lose=0, win=1, draw=2)

In [ ]:
c.lose

In [ ]:
from functools import reduce
from funcy import compose

def to_counts(c):
    return Counts(win=c[1], lose=c[-1], draw=c[0])

def addcounts(c, acc):
    r = Counts(win=acc.win + c.win, lose=acc.lose + c.lose, draw=acc.draw + c.draw)
    return r

reduce(addcounts, map(to_counts, map(count, vps)), Counts(0, 0, 0))

In [ ]:
win, lose, draw = 0, 0, 0
acc = Counts(0, 0, 0)
for vp in vps:
    c = count(vp)
    win += c[1]
    lose += c[-1]
    draw += c[0]
    #print(win, lose, draw)
    acc = addcounts(to_counts(c), acc)
win, lose, draw

In [ ]:
fig, ax = plt.subplots(1, 2)
plt.title(f'{win}, {lose}, {draw}')
plt.sca(ax[0])
draw_pdf_contours(Dirichlet((c[1], c[-1], c[0])))
plt.sca(ax[1])
draw_pdf_contours(Dirichlet((win, lose, draw)))
plt.show()

In [ ]:
c = reduce(addcounts, map(to_counts, map(count, vps)), Counts(0, 0, 0))
draw_pdf_contours(Dirichlet((c.win, c.lose, c.draw)))

In [ ]:
fig, ax = plt.subplots(1, 2)
for i in range(2):
    c = reduce(addcounts, map(to_counts, map(count, filter(lambda vp: get_group(vp)==i, vps))), Counts(0, 0, 0))
    plt.sca(ax[i])
    draw_pdf_contours(Dirichlet((c.win, c.lose, c.draw)))

In [ ]:
for i in range(2):
    c = reduce(addcounts, map(to_counts, map(count, filter(lambda vp: get_group(vp)==i, vps))), Counts(0, 0, 0))
    print(c)

In [ ]:
for i in range(2):
    c = reduce(addcounts, map(to_counts, map(count, filter(lambda vp: get_group(vp)==i, vps))), Counts(0, 0, 0))
    draw_pdf_contours(Dirichlet((c.win, c.lose, c.draw)))
    plt.savefig(f'dirichlet_group{i}.png')